# Demo Scenario 2
> Validate Demonstrate Scenario 2 on a notebook

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| eval: false

from llmcam.utils.downloader import *
from llmcam.vision.dtcam import *
from llmcam.utils.file_manager import *
from llmcam.core.fn_to_fc import *
from llmcam.utils.store import *
from llmcam.vision.yolo import *
from llmcam.vision.plotting import *
from llmcam.vision.gpt4v import *

In [ ]:
#| eval: false

def send_notification(
        msg: str  # The message to send
    ):
    """Send a notification"""
    global messages
    messages.append(form_msg("assistant", f"Notification: {msg}"))
    complete(messages, tools)
    return msg

from threading import Thread
import time

def start_notifications_stream(
    messages: list  # Previous conversation with the user
):
    """Start the notifications stream and monitoring. At the end of the stream, notify users about the results."""
    global tools

    # Add sending notification to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notifications_stream' ]
    subtools.append(tool_schema(send_notification, 'send_notification'))

    # Copy the messages to avoid modifying the original list
    messages = [ message for message in messages ]

    # Define the functionality to use
    def monitoring_loop():
        for _ in range(5):  
            time.sleep(10)  

            # Ask GPT if the condition is met
            messages.append({
                "role": "system",
                "content": "Evaluate the data. Does it satisfy the condition stated by the user? When the condition is met, send a message to user only if the notification has not been sent before! "
            })
            complete(messages, tools=subtools)

    # Start the notifications stream
    t = Thread(target=monitoring_loop)
    t.start()

    return 'Notifications stream started'

In [ ]:
#| eval: false
tools = []
def execute_handler(function_name, **kwargs):
    execute_handler_core(tools, function_name, **kwargs)

tools.extend([handler_schema(function, service_name="toolbox_handler", fixup=execute_handler) for function in [
        add_api_tools,
        add_function_tools,
        remove_tools,
        capture_youtube_live_frame_and_save,
        ask_gpt4v_about_image_file,
        list_image_files,
        list_detection_files,
        plot_object,
        detect_objects
    ]])

tools.append(tool_schema(start_notifications_stream, 'notification'))
tools[-1]['function']['parameters'] = {
    'type': 'object',
    'properties': {
        'messages': {
            'type': 'array',
            'items': {
                'type': 'object',
                'properties': {
                    'role': {
                        'type': 'string',
                        'enum': ['user', 'tool', 'system', 'assistant']
                    },
                    'content': {
                        'type': 'string'
                    }
                }
            }
        }
    }
}

In [ ]:
#| eval: false

messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "What can LLMCAM do?")
])
# messages.append(form_msg("user", "Here is the live feed for Santa Claus Village: https://www.youtube.com/watch?v=Cp4RRAEgpeU to capture"))
# messages.append(form_msg("user", "How many people are there currently?"))
# messages.append(form_msg("user", "Show me the image path of your detection."))
# messages.append(form_msg("user", "That is the image, not the detection file. I want the detection path."))
# messages.append(form_msg("user", "List the images captured at Santa Claus Village during yesterday and today."))
# messages.append(form_msg("user", "Plot the number of people in those images using YOLO and GPT."))
# messages.append(form_msg("user", "Monitor the live feed at Santa Claus Village (https://www.youtube.com/watch?v=Cp4RRAEgpeU) and notify me when there are fewer than 20 people."))
complete(messages, tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
What can LLMCAM do?
>> Assistant:
LLMCAM is a tool that captures images from YouTube Live streams and performs image analysis tasks.
Here's what it can do:  - **Capture YouTube Live Frame**: It can capture a JPEG file from a YouTube
Live stream and save it in a data directory for further processing.    - **List Captured Images**:
It can list all captured image files, which are named based on the date, time, and place of capture.
- **List Detection Files**: It can list all detection image files that start with "detection_" in
their name.  - **Object Detection**: It can run YOLO (You Only Look Once) object detection on an
input image to identify objects present.  - **Plot Object Occurrences**: It can generate a bar plot
showing the number of instances of a specified object detected in a list of images.  - **Ask GPT-4V
about Images**: It can use GPT-4V to provide quantitative information a

1. User: What Function Calls (FCs) do you have?
2. Assistant: I can monitor YouTube Live feeds and integrate with various services to provide insights and notifications.
3. User: Here’s the YouTube Live feed for Santa Claus Village.
4. Assistant: The YouTube Live feed for Santa Claus Village has been registered.
5. User: How many people are currently there?
6. Assistant: There are currently 52 people at Santa Claus Village.
7. User: Plot the change in the number of people at Santa Claus Village over the last 3 hours.
8. Assistant: [Generates and displays a bar chart showing the trend in people count over the last 3 hours.]
9. User: Notify me if there are fewer than 10 people.
10. Assistant: Notification rule set. I’ll alert you when the number of people at Santa Claus Village drops below 10.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()